In [3]:
import pandas as pd
import numpy as np
import datetime
import covidmap
from covidmap.preprocess.clean_df import choose_columns, choose_granularity
from covidmap.io.load_db import Load_db
import plotly.express as px

ModuleNotFoundError: No module named 'covidmap'

In [102]:
# Load data from covidmap (because already set)
df = choose_columns(Load_db.save_as_df(), ['date', 'granularite', 'maille_code', 'maille_nom', 'reanimation', 'source_nom'])

In [103]:
# Generate a first dataframe of ICU (Intensive Care Unit) by department
df_dep = df.copy()
df_dep = choose_granularity(df_dep, 'departement')
df_dep.drop('granularite', axis=1, inplace=True)
df_dep.loc[df_dep['source_nom'] == "Santé publique France Data", :]
df_dep.drop(['source_nom'], 1, inplace=True)
df_dep = df_dep.set_index('date')
df_dep = df_dep.sort_index()
df_dep.index = pd.to_datetime(df_dep.index)
df_dep.fillna(0, inplace=True)
df_dep['reanimation'] = df_dep['reanimation'].astype(int)

In [104]:
df_dep

,maille_code,maille_nom,reanimation
date,,,
2020-01-24,DEP-16,Charente,0
2020-01-24,DEP-17,Charente-Maritime,0
2020-01-24,DEP-19,Corrèze,0
2020-01-24,DEP-23,Creuse,0
2020-01-24,DEP-24,Dordogne,0
...,...,...,...
2021-04-16,DEP-971,Guadeloupe,21
2021-04-16,DEP-972,Martinique,31
2021-04-16,DEP-973,Guyane,13


In [105]:
dict_dep = {}
for department in df_dep['maille_nom'].unique().tolist():
    dict_dep[department] = pd.DataFrame(df_dep.loc[df_dep['maille_nom']== department,'reanimation']).resample("1D").sum()
    dict_dep[department] = dict_dep[department].rename(columns = {"reanimation": department})

In [106]:
df_all_dep = pd.DataFrame()
for department in df_dep['maille_nom'].unique().tolist() :
    df_all_dep = pd.concat([df_all_dep, dict_dep[department]], axis=1)

df_all_dep.fillna(0, inplace=True)
df_all_dep = df_all_dep.astype(int)
df_all_dep = df_all_dep.reset_index()

In [107]:
df_all_dep.head()

,date,Charente,Charente-Maritime,Corrèze,Creuse,Dordogne,Gironde,Landes,Lot-et-Garonne,Pyrénées-Atlantiques,...,Orne,Alpes-de-Haute-Provence,Gers,Hautes-Pyrénées,Guadeloupe,La Réunion,Pyrénées-Orientales,Lozère,Mayotte,Ariège
0,2020-01-24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-01-25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2020-01-26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020-01-27,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2020-01-28,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
def icu_dep_all():
    fig1 = px.line(
        df_all_dep,
        x='date', y=df_all_dep.columns,
        range_x=['2020-03-17',datetime.datetime.today().strftime('%Y-%m-%d')],
        title='Intensive care beds occupied since 1st confinement in french departments', 
        height=500, width=800)
    fig1.update_xaxes(dtick='M1', tickformat="%d\n%b")
    return(fig1.show())


def icu_dep_conf1():
    fig2 = px.line(
        df_all_dep,
        x='date', y=df_all_dep.columns,
        range_x=['2020-03-17','2020-05-10'],
        title='Intensive care beds occupied during the 1st confinement in french departments', 
        height=500, width=800)
    fig2.update_xaxes(dtick='M1', tickformat="%d\n%b")
    return(fig2.show)


def icu_dep_dec():
    fig3 = px.line(
        df_all_dep,
        x='date', y=df_all_dep.columns,
        range_x=['2020-05-11','2020-10-29'],
        title='Intensive care beds occupied during deconfinement in french departments', 
        height=500, width=800)
    fig3.update_xaxes(dtick='M1', tickformat="%d\n%b")
    return(fig3.show())


def icu_dep_conf2():
    fig4 = px.line(
        df_all_dep,
        x='date', y=df_all_dep.columns,
        range_x=['2020-10-30','2021-12-14'],
        title='Intensive care beds occupied during the 2nd confinement in french departments', 
        height=500, width=800)
    fig4.update_xaxes(dtick='M1', tickformat="%d\n%b")
    return(fig4.show())


def icu_dep_curfew():
    fig5 = px.line(
        df_all_dep,
        x='date', y=df_all_dep.columns,
        range_x=['2020-12-15','2021-04-02'],
        title='Intensive care beds occupied during curfew in french departments', 
        height=500, width=800)
    fig5.update_xaxes(dtick='M1', tickformat="%d\n%b")
    return(fig5.show())


def icu_dep_conf3():
    fig6 = px.line(
        df_all_dep,
        x='date', y=df_all_dep.columns,
        range_x=['2021-04-03',datetime.datetime.today().strftime('%Y-%m-%d')],
        title='Intensive care beds occupied during the 3rd confinement in french departments', 
        height=500, width=800)
    return(fig6.show())

In [109]:
icu_dep_conf3()

In [110]:
# Generate a first dataframe of ICU (Intensive Care Unit) by reg
df_reg = df.copy()
df_reg = choose_granularity(df_reg, 'region')
df_reg.drop(['granularite', 'maille_code'], axis=1, inplace=True)
df_reg.loc[df_reg['source_nom'] == "OpenCOVID19-fr", :]
df_reg.drop(['source_nom'], 1, inplace=True)
df_reg = df_reg.set_index('date')
df_reg = df_reg.sort_index()
df_reg = df_reg.loc['2020-03-17':]
df_reg.index = pd.to_datetime(df_reg.index)
df_reg.fillna(0, inplace=True)
df_reg['reanimation'] = df_reg['reanimation'].astype(int)
df_reg

,maille_nom,reanimation
date,,
2020-03-17,Guadeloupe,2
2020-03-17,Guadeloupe,0
2020-03-17,Martinique,0
2020-03-17,Guyane,0
2020-03-17,La Réunion,0
...,...,...
2021-04-16,Nouvelle-Aquitaine,259
2021-04-16,Occitanie,405
2021-04-16,Auvergne-Rhône-Alpes,688


In [111]:
dict_reg = {}
for region in df_reg['maille_nom'].unique().tolist():
    dict_reg[region] = pd.DataFrame(df_reg.loc[df_reg['maille_nom']== region,'reanimation']).resample("1D").sum()
    dict_reg[region] = dict_reg[region].rename(columns = {"reanimation": region})

In [112]:
df_all_reg = pd.DataFrame()
for region in df_reg['maille_nom'].unique().tolist() :
    df_all_reg = pd.concat([df_all_reg, dict_reg[region]], axis=1)

df_all_reg.fillna(0, inplace=True)
df_all_reg = df_all_reg.astype(int)
df_all_reg = df_all_reg.reset_index()

In [113]:
df_all_reg.head()

,date,Guadeloupe,Martinique,Guyane,La Réunion,Mayotte,Île-de-France,Centre-Val de Loire,Bourgogne-Franche-Comté,Normandie,Hauts-de-France,Grand Est,Pays de la Loire,Bretagne,Nouvelle-Aquitaine,Occitanie,Auvergne-Rhône-Alpes,Provence-Alpes-Côte d'Azur,Corse
0,2020-03-17,2,0,0,0,0,0,0,31,0,0,0,13,0,0,32,0,22,0
1,2020-03-18,0,5,0,2,0,294,5,76,4,87,204,17,48,12,76,35,57,0
2,2020-03-19,3,12,0,2,0,360,10,107,6,105,541,34,52,11,117,79,88,0
3,2020-03-20,4,12,0,2,1,484,23,144,13,136,646,42,52,15,133,87,114,14
4,2020-03-21,5,5,0,2,1,545,29,158,21,144,767,21,50,28,142,88,141,14


In [114]:
def icu_reg_all():
    fig1 = px.line(
        df_all_reg,
        x='date', y=df_all_reg.columns,
        range_x=['2020-03-17',datetime.datetime.today().strftime('%Y-%m-%d')],
        title='Intensive care beds occupied since 1st confinement in french regions', 
        height=500, width=800)
    fig1.update_xaxes(dtick='M1', tickformat="%d\n%b")
    return(fig1.show())


def icu_reg_conf1():
    fig2 = px.line(
        df_all_reg,
        x='date', y=df_all_reg.columns,
        range_x=['2020-03-17','2020-05-10'],
        title='Intensive care beds occupied during the 1st confinement in french regions', 
        height=500, width=800)
    fig2.update_xaxes(dtick='M1', tickformat="%d\n%b")
    return(fig2.show)


def icu_reg_dec():
    fig3 = px.line(
        df_all_reg,
        x='date', y=df_all_reg.columns,
        range_x=['2020-05-11','2020-10-29'],
        title='Intensive care beds occupied during deconfinement in french regions', 
        height=500, width=800)
    fig3.update_xaxes(dtick='M1', tickformat="%d\n%b")
    return(fig3.show())


def icu_reg_conf2():
    fig4 = px.line(
        df_all_reg,
        x='date', y=df_all_reg.columns,
        range_x=['2020-10-30','2021-12-14'],
        title='Intensive care beds occupied during the 2nd confinement in french regions', 
        height=500, width=800)
    fig4.update_xaxes(dtick='M1', tickformat="%d\n%b")
    return(fig4.show())


def icu_reg_curfew():
    fig5 = px.line(
        df_all_reg,
        x='date', y=df_all_reg.columns,
        range_x=['2020-12-15','2021-04-02'],
        title='Intensive care beds occupied during curfew in french regions', 
        height=500, width=800)
    fig5.update_xaxes(dtick='M1', tickformat="%d\n%b")
    return(fig5.show())


def icu_reg_conf3():
    fig6 = px.line(
        df_all_reg,
        x='date', y=df_all_reg.columns,
        range_x=['2021-04-03',datetime.datetime.today().strftime('%Y-%m-%d')],
        title='Intensive care beds occupied during the 3rd confinement in french regions', 
        height=500, width=800)
    return(fig6.show())

In [115]:
icu_reg_conf3()

In [116]:
icu_bretagne = pd.concat([df_all_dep['date'], df_all_dep['Finistère'], df_all_dep["Côtes-d'Armor"], df_all_dep["Morbihan"], df_all_dep["Ille-et-Vilaine"]], axis=1)

icu_normandie = pd.concat([df_all_dep['date'], df_all_dep['Manche'], df_all_dep["Calvados"], df_all_dep["Eure"], df_all_dep["Orne"], df_all_dep["Seine-Maritime"]], axis=1)

icu_pays_de_la_loire = pd.concat([df_all_dep['date'], df_all_dep['Mayenne'], df_all_dep["Sarthe"], df_all_dep["Vendée"], df_all_dep["Loire-Atlantique"], df_all_dep["Maine-et-Loire"]], axis=1)

icu_centre_val_de_loire = pd.concat([df_all_dep['date'], df_all_dep['Cher'], df_all_dep["Eure-et-Loir"], df_all_dep["Indre"], df_all_dep["Indre-et-Loire"], df_all_dep["Loir-et-Cher"], df_all_dep["Loiret"]], axis=1)

icu_bourgogne_franche_comte = pd.concat([df_all_dep['date'], df_all_dep["Côte-d'Or"], df_all_dep["Doubs"], df_all_dep["Jura"], df_all_dep["Nièvre"], df_all_dep["Haute-Saône"], df_all_dep["Saône-et-Loire"], df_all_dep["Yonne"]], axis=1)

icu_grand_est = pd.concat([df_all_dep['date'], df_all_dep["Ardennes"], df_all_dep["Aube"], df_all_dep["Marne"], df_all_dep["Haute-Marne"], df_all_dep["Meurthe-et-Moselle"], df_all_dep["Meuse"], df_all_dep["Moselle"], df_all_dep["Bas-Rhin"], df_all_dep["Haut-Rhin"], df_all_dep["Vosges"]], axis=1)

icu_hauts_de_france = pd.concat([df_all_dep['date'], df_all_dep["Aisne"], df_all_dep["Nord"], df_all_dep["Oise"], df_all_dep["Pas-de-Calais"], df_all_dep["Somme"]], axis=1)

icu_ile_de_france = pd.concat([df_all_dep['date'], df_all_dep["Paris"], df_all_dep["Seine-et-Marne"], df_all_dep["Yvelines"], df_all_dep["Essonne"], df_all_dep["Hauts-de-Seine"], df_all_dep["Seine-Saint-Denis"], df_all_dep["Val-de-Marne"], df_all_dep["Val-d'Oise"]], axis=1)

icu_nouvelle_aquitaine = pd.concat([df_all_dep['date'], df_all_dep["Charente"], df_all_dep["Charente-Maritime"], df_all_dep["Corrèze"], df_all_dep["Creuse"], df_all_dep["Dordogne"], df_all_dep["Gironde"], df_all_dep["Landes"], df_all_dep["Lot-et-Garonne"], df_all_dep["Pyrénées-Atlantiques"], df_all_dep["Deux-Sèvres"], df_all_dep["Vienne"], df_all_dep["Haute-Vienne"]], axis=1)

icu_auvergne_rhone_alpes = pd.concat([df_all_dep['date'], df_all_dep["Ain"], df_all_dep["Allier"], df_all_dep["Ardèche"], df_all_dep["Cantal"], df_all_dep["Drôme"], df_all_dep["Isère"], df_all_dep["Loire"], df_all_dep["Haute-Loire"], df_all_dep["Puy-de-Dôme"], df_all_dep["Rhône"], df_all_dep["Savoie"], df_all_dep["Haute-Savoie"]], axis=1)

icu_occitanie = pd.concat([df_all_dep['date'], df_all_dep["Ariège"], df_all_dep["Aude"], df_all_dep["Aveyron"], df_all_dep["Gard"], df_all_dep["Haute-Garonne"], df_all_dep["Gers"], df_all_dep["Hérault"], df_all_dep["Lot"], df_all_dep["Lozère"], df_all_dep["Hautes-Pyrénées"], df_all_dep["Pyrénées-Orientales"], df_all_dep["Tarn"], df_all_dep["Tarn-et-Garonne"]], axis=1)

icu_paca = pd.concat([df_all_dep['date'], df_all_dep["Alpes-de-Haute-Provence"], df_all_dep["Hautes-Alpes"], df_all_dep["Alpes-Maritimes"], df_all_dep["Bouches-du-Rhône"], df_all_dep["Var"], df_all_dep["Vaucluse"]], axis=1)

icu_corse = pd.concat([df_all_dep['date'], df_all_dep["Corse-du-Sud"], df_all_dep["Haute-Corse"]], axis=1)

icu_dom = pd.concat([df_all_dep['date'], df_all_dep["Guadeloupe"], df_all_dep["Martinique"], df_all_dep["Guyane"], df_all_dep["La Réunion"], df_all_dep["Mayotte"]], axis=1)

In [117]:
icu_by_reg = {}
icu_by_reg['Bretagne'] = icu_bretagne
icu_by_reg['Normandie'] = icu_normandie
icu_by_reg['Pays de la Loire'] = icu_pays_de_la_loire
icu_by_reg['Centre-Val-de-Loire'] = icu_centre_val_de_loire
icu_by_reg['Bourgogne-Franche-Comté'] = icu_bourgogne_franche_comte
icu_by_reg['Grand-Est'] = icu_grand_est
icu_by_reg['Hauts-de-France'] = icu_hauts_de_france
icu_by_reg['Île-de-France'] = icu_ile_de_france
icu_by_reg['Nouvelle-Aquitaine'] = icu_nouvelle_aquitaine
icu_by_reg['Auvergne-Rhône-Alpes'] = icu_auvergne_rhone_alpes
icu_by_reg['Occitanie'] = icu_occitanie
icu_by_reg["Provences-Alpes-Côte-d'Azur"] = icu_paca
icu_by_reg['Corse'] = icu_corse
icu_by_reg["Régions d'Outre Mer"] = icu_dom

In [118]:
def icu_by_reg_conf1(region):
    fig1 = px.line(
        icu_by_reg[f'{region}'],
        x='date', y=icu_by_reg[f'{region}'].columns,
        range_x=['2020-03-17',datetime.datetime.today().strftime('%Y-%m-%d')],
        title=f'Intensive care beds occupied since 1st confinement in {region}', 
        height=500, width=800)
    fig1.update_xaxes(dtick='M1', tickformat="%d\n%b")
    return(fig1.show())


def icu_by_reg_conf1(region):
    fig2 = px.line(
        icu_by_reg[f'{region}'],
        x='date', y=icu_by_reg[f'{region}'].columns,
        range_x=['2020-03-17','2020-05-10'],
        title=f'Intensive care beds occupied during the 1st confinement in {region}', 
        height=500, width=800)
    fig2.update_xaxes(dtick='M1', tickformat="%d\n%b")
    return(fig2.show)


def icu_by_reg_dec(region):
    fig3 = px.line(
        icu_by_reg[f'{region}'],
        x='date', y=icu_by_reg[f'{region}'].columns,
        range_x=['2020-05-11','2020-10-29'],
        title=f'Intensive care beds occupied during deconfinement in {region}', 
        height=500, width=800)
    fig3.update_xaxes(dtick='M1', tickformat="%d\n%b")
    return(fig3.show())


def icu_by_reg_conf2(region):
    fig4 = px.line(
        icu_by_reg[f'{region}'],
        x='date', y=icu_by_reg[f'{region}'].columns,
        range_x=['2020-10-30','2021-12-14'],
        title=f'Intensive care beds occupied during the 2nd confinement in {region}', 
        height=500, width=800)
    fig4.update_xaxes(dtick='M1', tickformat="%d\n%b")
    return(fig4.show())


def icu_by_reg_curfew(region):
    fig5 = px.line(
        icu_by_reg[f'{region}'],
        x='date', y=icu_by_reg[f'{region}'].columns,
        range_x=['2020-12-15','2021-04-02'],
        title=f'Intensive care beds occupied during curfew in {region}', 
        height=500, width=800)
    fig5.update_xaxes(dtick='M1', tickformat="%d\n%b")
    return(fig5.show())


def icu_by_reg_conf3(region):
    fig6 = px.line(
        icu_by_reg[f'{region}'],
        x='date', y=icu_by_reg[f'{region}'].columns,
        range_x=['2021-04-03',datetime.datetime.today().strftime('%Y-%m-%d')],
        title=f'Intensive care beds occupied during the 3rd confinement in {region}', 
        height=500, width=800)
    return(fig6.show())

In [119]:
icu_by_reg_conf3('Occitanie')

In [120]:
df_all_reg['Total'] = np.sum(df_all_reg.iloc[:,1:], axis=1)

In [121]:
def icu_all_reg():
    fig = px.bar(
        df_all_reg,
        x='date', y='Total',
        range_x=['2020-03-17',datetime.datetime.today().strftime('%Y-%m-%d')],
    title=f'patients in reanimation flux in France during Covid19 crisis',
    color='Total',
    labels={'Total':'Number of patients'},
    height=500, width=800)
    return(fig.show())


In [122]:
icu_all_reg()

In [123]:
df_all_reg["Régions d'Outre Mer"] = df_all_reg["Guadeloupe"]+df_all_reg['Martinique']+df_all_reg['Guyane']+df_all_reg['La Réunion']+df_all_reg['Mayotte']
df_all_reg_tot = df_all_reg['Total']
df_all_reg.drop(['Guadeloupe', 'Martinique', 'Guyane', 'La Réunion', 'Mayotte','Total'], axis=1, inplace=True)

df_all_reg.rename(columns={"Provence-Alpes-Côte d'Azur": "Provence-Alpes-Côte-d'Azur", "Centre-Val de Loire": 'Centre-Val-de-Loire', "Grand Est": "Grand-Est" }, inplace=True)

In [124]:
df_all_reg.head(0)

,date,Île-de-France,Centre-Val-de-Loire,Bourgogne-Franche-Comté,Normandie,Hauts-de-France,Grand-Est,Pays de la Loire,Bretagne,Nouvelle-Aquitaine,Occitanie,Auvergne-Rhône-Alpes,Provence-Alpes-Côte-d'Azur,Corse,Régions d'Outre Mer


In [125]:
icu_reg_total = pd.DataFrame(np.sum(df_all_reg.iloc[:,1:15], axis=0))
icu_reg_total.columns = ['Total number']
icu_reg_total

,Total number
Île-de-France,316089
Centre-Val-de-Loire,34585
Bourgogne-Franche-Comté,45671
Normandie,33113
Hauts-de-France,104864
Grand-Est,106549
Pays de la Loire,27361
Bretagne,18197
Nouvelle-Aquitaine,47004
Occitanie,68943


In [126]:
def icu_reg_repartition():
    fig = px.pie(
        icu_reg_total,
        values='Total number',
        names=icu_reg_total.index,
        title='Repartition of patients in ICU by region since the begining of Covid19 crisis',color_discrete_sequence=px.colors.sequential.RdBu,
        height=500, width=800)
    return(fig.show())

In [127]:
icu_reg_repartition()

In [1]:
icu_beds_reg = pd.read_csv('icu/bed_rea_reg.csv', delimiter=';')

NameError: name 'pd' is not defined

In [129]:
icu_beds_reg = icu_beds_reg.rename(columns={'Unnamed: 0' : 'Région'})
icu_beds_reg = icu_beds_reg.drop(['CHR', 'Autres'], 1)

In [130]:
icu_beds_reg = icu_beds_reg.loc[:13]
icu_beds_reg = icu_beds_reg.rename(columns={'Confondu' : 'Nombre de lits'})
icu_beds_reg = icu_beds_reg.reindex(index=[7, 8, 9, 4, 6, 5, 3, 1, 2, 0, 10, 11, 12, 13])
icu_beds_reg = icu_beds_reg.reset_index()
icu_beds_reg = icu_beds_reg.drop(['index'],1)
icu_beds_reg = icu_beds_reg.rename(index={"Provence-Alpes-Côte-d'Azur": "Provence-Alpes-Côte d'Azur" })


In [131]:
icu_beds_reg = icu_beds_reg.set_index('Région')

In [132]:
icu_beds_reg

,Nombre de lits
Région,
Île-de-France,1225
Normandie,258
Nouvelle-Aquitaine,443
Corse,18
Hauts-de-France,474
Grand-Est,505
Centre-Val-de-Loire,190
Bourgogne-Franche-Comté,203
Bretagne,171


In [133]:
icu_beds_reg_prop = df_all_reg.copy()
for col in icu_beds_reg.index.tolist() :
    for row in range(len(icu_beds_reg_prop)) :
        icu_beds_reg_prop.loc[row, col] = (icu_beds_reg_prop.loc[row, col]/int(icu_beds_reg.loc[col,'Nombre de lits']))*100


In [134]:
icu_beds_reg_prop

,date,Île-de-France,Centre-Val-de-Loire,Bourgogne-Franche-Comté,Normandie,Hauts-de-France,Grand-Est,Pays de la Loire,Bretagne,Nouvelle-Aquitaine,Occitanie,Auvergne-Rhône-Alpes,Provence-Alpes-Côte-d'Azur,Corse,Régions d'Outre Mer
0,2020-03-17,0.000000,0.000000,15.270936,0.000000,0.000000,0.000000,6.565657,0.000000,0.000000,6.639004,0.000000,4.489796,0.000000,1.234568
1,2020-03-18,24.000000,2.631579,37.438424,1.550388,18.354430,40.396040,8.585859,28.070175,2.708804,15.767635,5.700326,11.632653,0.000000,4.320988
2,2020-03-19,29.387755,5.263158,52.709360,2.325581,22.151899,107.128713,17.171717,30.409357,2.483070,24.273859,12.866450,17.959184,0.000000,10.493827
3,2020-03-20,39.510204,12.105263,70.935961,5.038760,28.691983,127.920792,21.212121,30.409357,3.386005,27.593361,14.169381,23.265306,77.777778,11.728395
4,2020-03-21,44.489796,15.263158,77.832512,8.139535,30.379747,151.881188,10.606061,29.239766,6.320542,29.460581,14.332248,28.775510,77.777778,8.024691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,2021-04-12,143.510204,104.210526,94.088670,90.310078,147.046414,98.613861,76.767677,80.701754,58.916479,87.551867,110.260586,115.918367,94.444444,65.432099
392,2021-04-13,145.061224,107.894737,89.162562,89.147287,148.734177,102.970297,71.717172,79.532164,59.819413,86.099585,111.074919,116.734694,94.444444,64.814815
393,2021-04-14,145.632653,105.263158,88.177340,86.821705,148.312236,99.801980,72.222222,81.286550,56.884876,82.780083,110.912052,117.551020,88.888889,62.962963
394,2021-04-15,144.734694,108.421053,86.699507,85.271318,149.789030,99.405941,72.222222,83.625731,58.013544,84.854772,113.029316,116.938776,88.888889,62.962963


In [137]:
def heat_map_icu_reg():
    fig = px.imshow(
        icu_beds_reg_prop.iloc[:,1:],
        y=icu_beds_reg_prop['date'],
        title='Regional saturation in % in ICU during Covid19 crisis',
        height=700, width=800)
    return(fig.show())

In [138]:
heat_map_icu_reg()